In [284]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.preprocessing import MinMaxScaler
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import time

In [2]:
scaler = MinMaxScaler()

In [3]:
# 자치구별 임대료 / 면적 데이터
seoul_district = pd.read_csv("C:/Users/ITSC/Desktop/MiniPrj2/Git/Data/seoul_district.csv")

In [4]:
# 자치구별 유동인구 종합 데이터
float_pop_data = pd.read_csv("C:/Users/ITSC/Desktop/MiniPrj2/Git/Data/유동인구종합.csv")

In [5]:
# 자치구별 건물연식 종합 데이터
district_age_data = pd.read_csv("C:/Users/ITSC/Desktop/MiniPrj2/Git/Data/district_age_data.csv")

In [6]:
# 자치구별 연령대 인구 데이터
seoul_pop_data = pd.read_csv("C:/Users/ITSC/Desktop/MiniPrj2/Git/Data/seoul_pop.csv")

In [7]:
# 자치구별 소득 데이터
seoul_income_data = pd.read_csv("C:/Users/ITSC/Desktop/MiniPrj2/Git/Data/seoul_income.csv")

In [8]:
# 자치구별 종사자, 사업체 수 데이터
employee_data = pd.read_csv("C:/Users/ITSC/Desktop/MiniPrj2/Git/Data/employee_data.csv")

In [9]:
# 자치구별 스타벅스 매장 수 데이터
starbucks_data = pd.read_csv("C:/Users/ITSC/Desktop/MiniPrj2/Git/Data/starbucks_data.csv")

## 임대료 데이터 요약 전처리

In [10]:
seoul_district = seoul_district.iloc[:, [0, 1, 3, 10, 12]]

In [11]:
seoul_district = seoul_district.rename(columns = {"Unnamed: 0" : "index"})

In [12]:
seoul_district = seoul_district.iloc[:, 1:].reset_index()

In [13]:
seoul_district

,index,자치구,중대형 1층,소형 1층,면적 (km²)
0,0,강남구,64.54,64.67,39.50
1,1,강동구,47.20,56.10,24.59
2,2,강북구,45.93,42.60,23.60
3,3,강서구,35.20,28.70,41.45
4,4,관악구,72.80,65.75,29.57
5,5,광진구,52.55,39.40,17.06
6,6,구로구,45.20,47.10,20.12
7,7,금천구,33.20,31.20,13.02
8,8,노원구,29.60,26.00,35.44
9,9,동대문구,44.32,42.40,14.22


In [99]:
seoul_district2 = pd.DataFrame(scaler.fit_transform(seoul_district.iloc[:, [2, 3]]))

In [102]:
seoul_district2 = seoul_district2.rename(columns = {0 : "중대형 정규화", 1 : "소형 정규화"})

In [104]:
seoul_district2["자치구"] = seoul_district["자치구"]

In [107]:
seoul_district2 = seoul_district2.iloc[:, [2, 0, 1]]

In [108]:
seoul_district2

,자치구,중대형 정규화,소형 정규화
0,강남구,0.615358,0.708242
1,강동구,0.309968,0.551282
2,강북구,0.287601,0.304029
3,강서구,0.098626,0.049451
4,관악구,0.760831,0.728022
5,광진구,0.404192,0.245421
6,구로구,0.274745,0.386447
7,금천구,0.063403,0.095238
8,노원구,0.000000,0.000000
9,동대문구,0.259246,0.300366


## 유동인구를 근거로 상권 임대료 평가

In [14]:
float_pop_data2 = float_pop_data.iloc[:, 12].reset_index()

In [15]:
float_pop_data2 = pd.merge(seoul_district, float_pop_data2, on = "index", how = "inner")

In [16]:
float_pop_data2.drop(21)

,index,자치구,중대형 1층,소형 1층,면적 (km²),종합/면적
0,0,강남구,64.54,64.67,39.50,1265358.0
1,1,강동구,47.20,56.10,24.59,524493.0
2,2,강북구,45.93,42.60,23.60,643757.0
3,3,강서구,35.20,28.70,41.45,515846.0
4,4,관악구,72.80,65.75,29.57,794169.0
5,5,광진구,52.55,39.40,17.06,991992.0
6,6,구로구,45.20,47.10,20.12,1089458.0
7,7,금천구,33.20,31.20,13.02,956760.0
8,8,노원구,29.60,26.00,35.44,545175.0
9,9,동대문구,44.32,42.40,14.22,1238371.0


In [17]:
float_pop_data3 = pd.concat([float_pop_data2.drop(21).reset_index(drop = True), pd.DataFrame(scaler.fit_transform(float_pop_data2.drop(21).iloc[:, 2:]))], axis = 1)

In [18]:
float_pop_data3 = float_pop_data3.rename(columns = {0 : "중대형 정규화", 1 : "소형 정규화", 3 : "유동 정규화"}).iloc[:, [1, 2, 3, 4, 5, 6, 7, 9]]

In [19]:
float_pop_data3["중대형 평가"] = float_pop_data3["유동 정규화"] - float_pop_data3["중대형 정규화"]
float_pop_data3["소형 평가"] = float_pop_data3["유동 정규화"] - float_pop_data3["소형 정규화"]

In [160]:
float_pop_data3.sort_values(by = "중대형 평가", ascending = False).reset_index(drop = True)

,자치구,중대형 1층,소형 1층,면적 (km²),종합/면적,중대형 정규화,소형 정규화,유동 정규화,중대형 평가,소형 평가
0,영등포구,34.60,35.60,24.55,1292575.0,0.113456,0.231158,0.893392,0.779936,0.662233
1,동대문구,44.32,42.40,14.22,1238371.0,0.334014,0.394895,0.831046,0.497032,0.436151
2,마포구,45.32,50.92,23.85,1219940.0,0.356705,0.600048,0.809847,0.453142,0.209799
3,동작구,53.75,35.35,16.36,1385262.0,0.547992,0.225138,1.000000,0.452008,0.774862
4,금천구,33.20,31.20,13.02,956760.0,0.081688,0.125211,0.507138,0.425450,0.381927
5,용산구,40.60,45.90,21.87,1039013.0,0.249603,0.479172,0.601745,0.352142,0.122574
6,성동구,44.55,48.00,16.82,1089352.0,0.339233,0.529738,0.659645,0.320412,0.129908
7,구로구,45.20,47.10,20.12,1089458.0,0.353982,0.508066,0.659767,0.305785,0.151701
8,송파구,39.97,32.50,33.88,969803.0,0.235307,0.156513,0.522140,0.286833,0.365627
9,양천구,33.80,35.10,17.41,827074.0,0.095303,0.219119,0.357974,0.262671,0.138855


In [161]:
float_pop_data4 = float_pop_data3.sort_values(by = "중대형 평가", ascending = False).reset_index(drop = True)

In [166]:
float_pop_data4 = float_pop_data4.iloc[:, [0, -2, -1]]

In [248]:
float_pop_data3.sort_values(by = "중대형 평가", ascending = False).reset_index(drop = True).iloc[:, [0, 4, 5, 6, 7, 8, 9]]

,자치구,종합/면적,중대형 정규화,소형 정규화,유동 정규화,중대형 평가,소형 평가
0,영등포구,1292575.0,0.113456,0.231158,0.893392,0.779936,0.662233
1,동대문구,1238371.0,0.334014,0.394895,0.831046,0.497032,0.436151
2,마포구,1219940.0,0.356705,0.600048,0.809847,0.453142,0.209799
3,동작구,1385262.0,0.547992,0.225138,1.000000,0.452008,0.774862
4,금천구,956760.0,0.081688,0.125211,0.507138,0.425450,0.381927
5,용산구,1039013.0,0.249603,0.479172,0.601745,0.352142,0.122574
6,성동구,1089352.0,0.339233,0.529738,0.659645,0.320412,0.129908
7,구로구,1089458.0,0.353982,0.508066,0.659767,0.305785,0.151701
8,송파구,969803.0,0.235307,0.156513,0.522140,0.286833,0.365627
9,양천구,827074.0,0.095303,0.219119,0.357974,0.262671,0.138855


## 자치구 건물 연식을 근거로 상권 임대료 평가

In [69]:
district_age_data2 = district_age_data.iloc[:, [0, 2]].rename(columns = {"Unnamed: 0" : "index"})

In [70]:
district_age_data2 = pd.merge(seoul_district, district_age_data2, on = "index", how = "inner")

In [71]:
district_age_data2 = district_age_data2.iloc[:, [0, 2, 3, 5]]

In [80]:
district_age_data2

,index,중대형 1층,소형 1층,연식
0,0,64.54,64.67,25.41
1,1,47.20,56.10,23.02
2,2,45.93,42.60,19.93
3,3,35.20,28.70,11.73
4,4,72.80,65.75,22.33
5,5,52.55,39.40,25.18
6,6,45.20,47.10,25.36
7,7,33.20,31.20,15.67
8,8,29.60,26.00,26.98
9,9,44.32,42.40,30.31


In [73]:
seoul_district.iloc[:, :2]

,index,자치구
0,0,강남구
1,1,강동구
2,2,강북구
3,3,강서구
4,4,관악구
5,5,광진구
6,6,구로구
7,7,금천구
8,8,노원구
9,9,동대문구


In [74]:
district_age_data3 = pd.concat([seoul_district.iloc[:, :2], pd.DataFrame(scaler.fit_transform(district_age_data2.iloc[:, 1:])).rename(columns = {0 : "중대형 정규화", 1 : "소형 정규화", 2 : "연식 정규화"})], axis = 1)

In [77]:
district_age_data3["중대형 평가"] = round(district_age_data3["연식 정규화"] - district_age_data3["중대형 정규화"], 4)
district_age_data3["소형 평가"] = round(district_age_data3["연식 정규화"] - district_age_data3["소형 정규화"], 4)

In [159]:
district_age_data3.sort_values(by = "중대형 평가", ascending = False).reset_index(drop = True)

,index,자치구,중대형 정규화,소형 정규화,연식 정규화,중대형 평가,소형 평가
0,8,노원구,0.000000,0.000000,0.712284,0.7123,0.7123
1,17,영등포구,0.088059,0.175824,0.785614,0.6976,0.6098
2,13,서초구,0.063403,0.256410,0.728631,0.6652,0.4722
3,18,용산구,0.193730,0.364469,0.851938,0.6582,0.4875
4,9,동대문구,0.259246,0.300366,0.867819,0.6086,0.5675
5,16,양천구,0.073970,0.166667,0.561420,0.4875,0.3948
6,6,구로구,0.274745,0.386447,0.636618,0.3619,0.2502
7,15,송파구,0.182635,0.119048,0.499299,0.3167,0.3803
8,11,마포구,0.276858,0.456410,0.548342,0.2715,0.0919
9,14,성동구,0.263297,0.402930,0.508641,0.2453,0.1057


## 자치구의 연령별 인구 비율을 근거로 임대료 평가

In [86]:
seoul_pop_data2 = seoul_pop_data.iloc[:, -7:-4]

In [89]:
seoul_pop_scaled = pd.DataFrame(scaler.fit_transform(seoul_pop_data2))

In [92]:
seoul_pop_scaled = seoul_pop_scaled.rename(columns = {0 : "30대", 1 : "40대", 2 : "50대"})

In [110]:
seoul_pop_data3 = pd.concat([seoul_district2, seoul_pop_scaled], axis = 1)

In [116]:
seoul_pop_data3["중대형 평가"] = seoul_pop_data3["중대형 정규화"] - 1 + seoul_pop_data3["50대"]
seoul_pop_data3["소형 평가"] = seoul_pop_data3["소형 정규화"] - 1 + seoul_pop_data3["50대"]

In [158]:
seoul_pop_data3.sort_values(by = "소형 평가").reset_index(drop = True)

,자치구,중대형 정규화,소형 정규화,30대,40대,50대,중대형 평가,소형 평가
0,강서구,0.098626,0.049451,0.517826,0.375531,0.301230,-0.600144,-0.649319
1,서초구,0.063403,0.256410,0.125609,0.211361,0.109979,-0.826619,-0.633611
2,노원구,0.000000,0.000000,0.302293,0.385209,0.427930,-0.572070,-0.572070
3,용산구,0.193730,0.364469,0.344997,0.211831,0.185940,-0.620330,-0.449591
4,영등포구,0.088059,0.175824,0.751367,0.452731,0.453112,-0.458829,-0.371064
5,강북구,0.287601,0.304029,0.223580,0.256902,0.337491,-0.374907,-0.358479
6,송파구,0.182635,0.119048,0.784161,0.693216,0.599630,-0.217735,-0.281322
7,금천구,0.063403,0.095238,0.772134,0.479652,0.657622,-0.278976,-0.247140
8,은평구,0.290595,0.267399,0.483080,0.459123,0.486181,-0.223224,-0.246420
9,종로구,0.776154,0.760623,0.000000,0.000000,0.000000,-0.223846,-0.239377


In [168]:
seoul_pop_data4 = seoul_pop_data3.sort_values(by = "소형 평가").reset_index(drop = True)

In [171]:
seoul_pop_data4 = seoul_pop_data4.iloc[:, [0, -2, -1]]

## 자치구별 소득 수준에 따른 임대료 평가

In [122]:
seoul_income_data2 = seoul_income_data.iloc[:, 2:]

In [133]:
seoul_income_data2

,지역내총생산(당해년가격) (백만원),1인당 지역내총생산 (천원)
0,77924019,155358
1,9364063,20717
2,3462961,11754
3,18347568,32709
4,6347562,12792
5,6554954,18732
6,14663398,34144
7,17801015,71770
8,6728358,13331
9,7711273,22164


In [128]:
seoul_income_data3 = pd.DataFrame(scaler.fit_transform(seoul_income_data2)).rename(columns = {0 : "지역내 총생산(100만)", 1 : "1인당 지역내 총생산(1천)"})

In [131]:
seoul_income_data4 = pd.concat([seoul_district2, seoul_income_data3], axis = 1)

In [132]:
seoul_income_data4

,자치구,중대형 정규화,소형 정규화,지역내 총생산(100만),1인당 지역내 총생산(1천)
0,강남구,0.615358,0.708242,1.000000,0.306682
1,강동구,0.309968,0.551282,0.079251,0.020368
2,강북구,0.287601,0.304029,0.000000,0.001308
3,강서구,0.098626,0.049451,0.199898,0.045869
4,관악구,0.760831,0.728022,0.038740,0.003515
5,광진구,0.404192,0.245421,0.041525,0.016147
6,구로구,0.274745,0.386447,0.150420,0.048920
7,금천구,0.063403,0.095238,0.192558,0.128932
8,노원구,0.000000,0.000000,0.043854,0.004661
9,동대문구,0.259246,0.300366,0.057054,0.023445


In [134]:
seoul_income_data4["중대형 평가"] = seoul_income_data4["1인당 지역내 총생산(1천)"] - seoul_income_data4["중대형 정규화"]
seoul_income_data4["소형 평가"] = seoul_income_data4["1인당 지역내 총생산(1천)"] - seoul_income_data4["중대형 정규화"]

In [156]:
seoul_income_data4.sort_values(by = "중대형 평가", ascending = False).reset_index(drop = True)

,자치구,중대형 정규화,소형 정규화,지역내 총생산(100만),1인당 지역내 총생산(1천),중대형 평가,소형 평가
0,영등포구,0.088059,0.175824,0.547977,0.208314,0.120255,0.120255
1,서초구,0.063403,0.256410,0.468335,0.182122,0.118719,0.118719
2,금천구,0.063403,0.095238,0.192558,0.128932,0.065529,0.065529
3,노원구,0.000000,0.000000,0.043854,0.004661,0.004661,0.004661
4,중구,1.000000,1.000000,0.777316,1.000000,0.000000,0.000000
5,강서구,0.098626,0.049451,0.199898,0.045869,-0.052758,-0.052758
6,양천구,0.073970,0.166667,0.052284,0.012244,-0.061725,-0.061725
7,용산구,0.193730,0.364469,0.132038,0.101602,-0.092128,-0.092128
8,송파구,0.182635,0.119048,0.381023,0.083255,-0.099380,-0.099380
9,중랑구,0.147939,0.375458,0.018765,0.003409,-0.144531,-0.144531


In [173]:
seoul_income_data5 = seoul_income_data4.sort_values(by = "중대형 평가", ascending = False).reset_index(drop = True)

In [176]:
seoul_income_data5 = seoul_income_data5.iloc[:, [0, -2, -1]]

## 종사자수/자치구별 인구 기준으로 상가 임대료 평가

In [141]:
employee_data2 = employee_data.loc[employee_data["자치구"].isin(seoul_district["자치구"])].reset_index(drop = True).iloc[:, [5, 8]]

In [145]:
employee_data2

,종사자밀도(A),종사자수/자치구별인구
0,11502.69,182.82
1,38810.63,293.31
2,6979.05,67.20
3,12053.51,71.36
4,7601.65,36.93
5,10297.20,40.68
6,6117.35,29.20
7,3399.17,27.38
8,3764.14,26.52
9,3669.78,23.16


In [148]:
employee_data3 = pd.concat([seoul_district2, pd.DataFrame(scaler.fit_transform(employee_data2)).rename(columns = {0 : "종사자 밀도", 1 : "종사자수/인구"})], axis = 1)

In [150]:
employee_data3["중대형 평가"] = employee_data3["종사자수/인구"] - employee_data3["중대형 정규화"]
employee_data3["소형 평가"] = employee_data3["종사자수/인구"] - employee_data3["중대형 정규화"]

In [157]:
employee_data3.sort_values(by = "중대형 평가", ascending = False).reset_index(drop = True)

,자치구,중대형 정규화,소형 정규화,종사자 밀도,종사자수/인구,중대형 평가,소형 평가
0,강동구,0.309968,0.551282,1.000000,1.000000,0.690032,0.690032
1,양천구,0.073970,0.166667,0.404438,0.319082,0.245112,0.245112
2,송파구,0.182635,0.119048,0.458039,0.306459,0.123825,0.123825
3,강서구,0.098626,0.049451,0.244394,0.178419,0.079793,0.079793
4,은평구,0.290595,0.267399,0.197319,0.352619,0.062024,0.062024
5,서초구,0.063403,0.256410,0.101895,0.103313,0.039910,0.039910
6,노원구,0.000000,0.000000,0.010307,0.012438,0.012438,0.012438
7,강남구,0.615358,0.708242,0.228839,0.591005,-0.024353,-0.024353
8,금천구,0.063403,0.095238,0.000000,0.015621,-0.047782,-0.047782
9,영등포구,0.088059,0.175824,0.092700,0.018064,-0.069995,-0.069995


In [179]:
employee_data4 = employee_data3.sort_values(by = "중대형 평가", ascending = False).reset_index(drop = True)

In [182]:
employee_data4 = employee_data4.iloc[:, [0, -2, -1]]

## 종합점수 매기기

In [202]:
float_pop_data4.iloc[:, :1].reset_index()

,index,자치구
0,0,영등포구
1,1,동대문구
2,2,마포구
3,3,동작구
4,4,금천구
5,5,용산구
6,6,성동구
7,7,구로구
8,8,송파구
9,9,양천구


In [203]:
seoul_pop_data4.iloc[:, :1].reset_index()

,index,자치구
0,0,강서구
1,1,서초구
2,2,노원구
3,3,용산구
4,4,영등포구
5,5,강북구
6,6,송파구
7,7,금천구
8,8,은평구
9,9,종로구


In [204]:
seoul_income_data5.iloc[:, :1].reset_index()

,index,자치구
0,0,영등포구
1,1,서초구
2,2,금천구
3,3,노원구
4,4,중구
5,5,강서구
6,6,양천구
7,7,용산구
8,8,송파구
9,9,중랑구


In [205]:
employee_data4.iloc[:, :1].reset_index()

,index,자치구
0,0,강동구
1,1,양천구
2,2,송파구
3,3,강서구
4,4,은평구
5,5,서초구
6,6,노원구
7,7,강남구
8,8,금천구
9,9,영등포구


In [209]:
final = pd.merge(seoul_pop_data4.iloc[:, :1].reset_index(), float_pop_data4.iloc[:, :1].reset_index(), on = "자치구", how = "left").fillna(11)

In [212]:
final = final.rename(columns = {"index_x" : "pop_rank", "index_y" : "floatpop_rank"})

In [216]:
final = pd.merge(final, seoul_income_data5.iloc[:, :1].reset_index(), on = "자치구", how = "inner").rename(columns = {"index" : "income_rank"})

In [220]:
final = pd.merge(final, employee_data4.iloc[:, :1].reset_index(), on = "자치구", how = "inner").rename(columns = {"index" : "employee_rank"})

In [224]:
final = final.iloc[:, [1, 0, 2, 3, 4]]

In [231]:
final["overrated_point"] = final.iloc[:, 1:].sum(axis = 1)

C:\Users\ITSC\AppData\Local\Temp\ipykernel_10976\3745164652.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final["overrated_point"] = final.iloc[:, 1:].sum(axis = 1)


In [236]:
final = final.sort_values(by = "overrated_point").reset_index(drop = True)

In [238]:
final.to_csv("./conclusion.csv")

In [239]:
final

,자치구,pop_rank,floatpop_rank,income_rank,employee_rank,overrated_point
0,영등포구,4,0.0,0,9,13.0
1,서초구,1,10.0,1,5,17.0
2,금천구,7,4.0,2,8,21.0
3,강서구,0,15.0,5,3,23.0
4,노원구,2,12.0,3,6,23.0
5,송파구,6,8.0,8,2,24.0
6,용산구,3,5.0,7,14,29.0
7,마포구,10,2.0,10,10,32.0
8,양천구,21,9.0,6,1,37.0
9,성동구,12,6.0,11,13,42.0


## 결론 검증 및 보강

In [252]:
seoul_sales = pd.read_csv("C:/Users/ITSC/Desktop/MiniPrj2/Git/Data/자치구별 매출액.csv", names = ["자치구", "매출(억)"])

In [253]:
seoul_sales

,자치구,매출(억)
0,강서구,4287
1,양천구,2613
2,구로구,3001
3,마포구,4410
4,영등포구,5231
5,금천구,1752
6,관악구,2547
7,동작구,1779
8,용산구,3225
9,서대문구,1771


In [259]:
verify01 = pd.merge(pd.merge(final, seoul_sales, on = "자치구", how = "left"), seoul_district.iloc[:, 1:], on = "자치구", how = "inner")

In [262]:
verify01["매출/(면적*임대료)"] = verify01["매출(억)"] / (verify01["면적 (km²)"] * verify01["중대형 1층"])

In [264]:
verify01.iloc[:, [0, 5, 6, -1]]

,자치구,overrated_point,매출(억),매출/(면적*임대료)
0,영등포구,13.0,5231,6.158247
1,서초구,17.0,8039,5.155175
2,금천구,21.0,1752,4.053079
3,강서구,23.0,4287,2.938233
4,노원구,23.0,2612,2.489934
5,송파구,24.0,6554,4.839816
6,용산구,29.0,3225,3.632076
7,마포구,32.0,4410,4.080001
8,양천구,37.0,2613,4.440419
9,성동구,42.0,2307,3.078746


In [266]:
url = "https://golmok.seoul.go.kr/stateArea.do"

page = urlopen(url)

soup = BeautifulSoup(page, "html.parser")

In [283]:
soup.select("tbody tr.branch.collapsed td")

[]

In [289]:
url = "https://golmok.seoul.go.kr/stateArea.do"

driver = webdriver.Chrome()
driver.get(url)

driver.implicitly_wait(2)

addr = driver.find_element(By.CSS_SELECTOR, "#stateArea > div.searchArea.forMo > div.swiper-container.swiper_infotype.swiper-container-horizontal.swiper-container-free-mode > div > div:nth-child(4)")
addr.click( ) 

driver.implicitly_wait(2)

addr2 = driver.find_element(By.CSS_SELECTOR, "#stateArea > div.searchArea.forMo > div.btn > div")
addr2.click( )

time.sleep(5)

soup = BeautifulSoup(driver.page_source, "lxml")

In [300]:
table = [i.text for i in soup.select("tbody tr.branch.collapsed td")]

In [308]:
table

['하위 메뉴 펼침종로구',
 '업종 전체',
 '81.9',
 '60.3',
 '49.2',
 '81.4',
 '61.3',
 '52.5',
 '82',
 '64.1',
 '54',
 '하위 메뉴 펼침중구',
 '업종 전체',
 '82.3',
 '60.3',
 '49.1',
 '79.2',
 '58.6',
 '48.1',
 '83',
 '64.6',
 '54.7',
 '하위 메뉴 펼침용산구',
 '업종 전체',
 '80.2',
 '58.1',
 '47.5',
 '80.5',
 '60.8',
 '49.6',
 '81.6',
 '63.1',
 '52.7',
 '하위 메뉴 펼침성동구',
 '업종 전체',
 '81.5',
 '61',
 '49.4',
 '80.1',
 '59.3',
 '48.7',
 '81.5',
 '62.6',
 '51.5',
 '하위 메뉴 펼침광진구',
 '업종 전체',
 '77.7',
 '52.5',
 '42.1',
 '77',
 '57.2',
 '45.2',
 '78.2',
 '58.5',
 '47.2',
 '하위 메뉴 펼침동대문구',
 '업종 전체',
 '78.7',
 '54.6',
 '43.5',
 '78.1',
 '55.9',
 '45.3',
 '76',
 '56.6',
 '46.8',
 '하위 메뉴 펼침중랑구',
 '업종 전체',
 '72.2',
 '48.1',
 '37.7',
 '74.9',
 '53.9',
 '43.3',
 '76',
 '55.3',
 '44.7',
 '하위 메뉴 펼침성북구',
 '업종 전체',
 '77',
 '52.3',
 '40.9',
 '77.6',
 '56.6',
 '45.8',
 '78.7',
 '59.4',
 '48.6',
 '하위 메뉴 펼침강북구',
 '업종 전체',
 '72.5',
 '47.7',
 '36.7',
 '73.6',
 '50.2',
 '40.9',
 '74.1',
 '52.3',
 '41.6',
 '하위 메뉴 펼침도봉구',
 '업종 전체',
 '74.6',
 '48.8',
 '38.7',


In [310]:
table2 = pd.DataFrame([table[i*11:i*11+11] for i in range(int(len(table)/11))])

In [313]:
table2 = table2.iloc[:, [0, 8, 9, 10]]

In [316]:
table2 = table2.rename(columns = {0 : "자치구", 8 : "1년생존", 9 : "3년생존", 10 : "5년생존"})

In [321]:
table2["자치구"] = table2["자치구"].str.slice(start = 8)

In [326]:
table2 = table2.sort_values("자치구").reset_index(drop = True)

In [335]:
table2["1년생존률평가"] = table2["1년생존"].astype(float)-79
table2["3년생존률평가"] = table2["3년생존"].astype(float)-59.6
table2["5년생존률평가"] = table2["5년생존"].astype(float)-48.9

In [336]:
table2

,자치구,1년생존,3년생존,5년생존,1년생존률평가,3년생존률평가,5년생존률평가
0,강남구,80.2,61.5,51.8,1.2,1.9,2.9
1,강동구,76.2,55.8,45.4,-2.8,-3.8,-3.5
2,강북구,74.1,52.3,41.6,-4.9,-7.3,-7.3
3,강서구,79.3,59.6,48.7,0.3,0.0,-0.2
4,관악구,76.3,55.8,43.5,-2.7,-3.8,-5.4
5,광진구,78.2,58.5,47.2,-0.8,-1.1,-1.7
6,구로구,76.9,59.3,48.6,-2.1,-0.3,-0.3
7,금천구,80.5,62,51.1,1.5,2.4,2.2
8,노원구,76.1,57.9,47.4,-2.9,-1.7,-1.5
9,도봉구,76.5,56.4,45.7,-2.5,-3.2,-3.2


In [338]:
verify02 = pd.merge(pd.merge(final, table2, on = "자치구", how = "left"), seoul_district.iloc[:, 1:], on = "자치구", how = "inner")

In [346]:
verify02.iloc[:, [0, 5, 9, 10, 11]]

,자치구,overrated_point,1년생존률평가,3년생존률평가,5년생존률평가
0,영등포구,13.0,1.0,1.3,1.1
1,서초구,17.0,2.9,3.7,4.3
2,금천구,21.0,1.5,2.4,2.2
3,강서구,23.0,0.3,0.0,-0.2
4,노원구,23.0,-2.9,-1.7,-1.5
5,송파구,24.0,0.0,0.2,0.0
6,용산구,29.0,2.6,3.5,3.8
7,마포구,32.0,0.4,-1.1,-1.9
8,양천구,37.0,-1.2,-2.5,-3.9
9,성동구,42.0,2.5,3.0,2.6
